In [2]:
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
from networkx.algorithms.components import strongly_connected_component_subgraphs
from networkx.algorithms.components import connected_component_subgraphs
import timeit
import os


import pickle
import time

workdir = "/home/michele/aalto/dm/DMproject/dmproject/approximate"
if(os.curdir != workdir):
    os.chdir("/home/michele/aalto/dm/DMproject/dmproject/approximate")

import sys
sys.path.insert(0, '/home/michele/aalto/dm/DMproject/dmproject/utils')

import utils

In [3]:
graph_files = []
graph_files.append(os.path.join(os.path.pardir, 'dataset', 'wiki_vote', 'Wiki-Vote.txt'))
graph_files.append(os.path.join(os.path.pardir, 'dataset', 'epinions', 'soc-Epinions1.txt'))
graph_files.append(os.path.join(os.path.pardir, 'dataset', 'gplus', 'gplus_combined.txt'))
graph_files.append(os.path.join(os.path.pardir, 'dataset', 'soc_pokec', 'soc-pokec-relationships.txt'))

In [5]:

'''
for file in graph_files:

    with open(file, "r") as fh:
        start_time = time.time()
        
        # import of directed
        dG = nx.read_adjlist(file, create_using=nx.DiGraph())
        # conversion to undirected graph
        G = dG.to_undirected()
        
        cc_lst = get_cc(G,file)
        allfiles_cc_lst.append(cc_lst)
        scc_lst = get_scc(dG,file)
        allfiles_scc_lst.append(scc_lst)
        
        largest_cc = get_largest_cc(cc_lst)
        allfiles_largest_cc_lst.append()
        largest_scc = get_largest_cc(scc_lst)
        allfiles_largest_cc_lst.append(largest_scc)
        
        # largest cc/scc len
        largest_cc_len = len(largest_cc)
        allfiles_largest_cc_len_lst.append(largest_cc_len)
        print("largest cc len completed")
        
        largest_scc_len = len(largest_scc)
        allfiles_largest_scc_len_lst.append(largest_scc_len)
        print("largest scc len completed")
        
        matrix_cc = reservoir_sampling(G, largest_cc_len, p_samples, n_repetitions)
        pickle.dump(matrix_cc, open(file + "_matrix_cc_list.pkl", "wb")) 
        matrix_scc = reservoir_sampling(G, largest_cc_len, p_samples, n_repetitions, directed=True)
        pickle.dump(matrix_scc, open(file + "_matrix_scc_list.pkl", "wb"))
        
        end_time = time.time()
        run_time = end_time - start_time
        allfiles_timing_lst.append(run_time)
        print(file, " - ", run_time)
'''

'\nfor file in graph_files:\n\n    with open(file, "r") as fh:\n        start_time = time.time()\n        \n        # import of directed\n        dG = nx.read_adjlist(file, create_using=nx.DiGraph())\n        # conversion to undirected graph\n        G = dG.to_undirected()\n        \n        cc_lst = get_cc(G,file)\n        allfiles_cc_lst.append(cc_lst)\n        scc_lst = get_scc(dG,file)\n        allfiles_scc_lst.append(scc_lst)\n        \n        largest_cc = get_largest_cc(cc_lst)\n        allfiles_largest_cc_lst.append()\n        largest_scc = get_largest_cc(scc_lst)\n        allfiles_largest_cc_lst.append(largest_scc)\n        \n        # largest cc/scc len\n        largest_cc_len = len(largest_cc)\n        allfiles_largest_cc_len_lst.append(largest_cc_len)\n        print("largest cc len completed")\n        \n        largest_scc_len = len(largest_scc)\n        allfiles_largest_scc_len_lst.append(largest_scc_len)\n        print("largest scc len completed")\n        \n        matr

In [15]:
file = graph_files[0]
fh = open(file, "r")


# import of directed
dG = nx.read_adjlist(file, create_using=nx.DiGraph())
# conversion to undirected graph
G = dG.to_undirected()




In [ ]:
cc_lst = utils.get_cc(G,file)
#allfiles_cc_lst.append(cc_lst)
scc_lst = utils.get_scc(dG,file)
#allfiles_scc_lst.append(scc_lst)

In [6]:
largest_cc = utils.get_largest_cc(cc_lst)
#allfiles_largest_cc_lst.append()
largest_scc = utils.get_largest_cc(scc_lst)
#allfiles_largest_cc_lst.append(largest_scc)

largest cc completed
largest cc completed


In [7]:
# largest cc/scc len
largest_cc_len = len(largest_cc)
#allfiles_largest_cc_len_lst.append(largest_cc_len)
print("largest cc len completed")
largest_scc_len = len(largest_scc)
#allfiles_largest_scc_len_lst.append(largest_scc_len)
print("largest scc len completed")

largest cc len completed
largest scc len completed


In [17]:
def reservoir_sampling(G, largest_cc_len, n_repetitions, p_samples, directed=False):
    res = []
    for p in p_samples:
        samples_mat_p = []
        for n in n_repetitions:
            samples_count = largest_cc_len*p
            samples_mat = []
            for i in range(n):
                samples = []
                
                if directed:
                    subG = nx.DiGraph()
                else:
                    subG = nx.Graph()
                
                for t, edge in enumerate(G.edges):
                    prob = np.random.uniform()
                    threshold = min(samples_count/(t+1), 1)
                    
                    if prob <= threshold: 
                        subG.add_edge(edge[0], edge[1])
                samples_mat.append(subG)
            samples_mat_p.append(samples_mat)
        res.append(samples_mat_p)
    return res

np.random.seed(132435)
p_samples = [0.2]#, 0.3, 0.4, 0.7]
n_repetitions = [2]#, 50, 100, 500, 1000]

matrix_cc = reservoir_sampling(G, largest_cc_len, n_repetitions, p_samples)
pickle.dump(matrix_cc, open(file + "_matrix_cc_list.pkl", "wb")) 
matrix_scc = reservoir_sampling(G, largest_cc_len, n_repetitions, p_samples, directed=True)
pickle.dump(matrix_scc, open(file + "_matrix_scc_list.pkl", "wb"))

In [19]:
newG = np.array(matrix_cc)
print(newG.shape)
print(len(newG[0,0,0].edges()))

(1, 1, 2)
7382
